In [444]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

path = "drive/MyDrive/Capstone/"
crop_1 = "Barley"
crop_2 = "Maize"
crop_6 = "Sorghum"
crop_7 = "Teff"
crop_8 = "Wheat"

year = "2021"

df = pd.read_excel(f"{path}Meher_{year}.xlsx")

In [445]:
df.columns


Index(['Region', 'Zone', 'District', 'Farmers Association', 'Enumeration Area',
       'Household Id', 'Head sex', 'Sampling weight', 'Field type', 'CROP',
       'Ownership type', 'Is field under Extension Program?',
       'Is Field Irrigated?', 'How many times is the field ploughed',
       'Orientation of the field', 'Is Field Prevented form Erosion',
       'If field Prevented form Erosion, common way of prevention',
       'Do youprevent land Degradation', 'Method to prevent land degredation',
       'Type of sowing', 'Seed / Seedling Type',
       'Quantity of improved seeds used', 'Quantity of indigenous seeds used',
       'Was crop damaged ?', 'Percent of damaged crop',
       'Prevention/precaution measure taken?', 'Type of measure if any?',
       'Is Fertilizer Used?', 'Type of fertilizer used if any?',
       'Type of chemical fertilizer', 'Quantity of UREA fertilizer used',
       'Quantity of DAP fertilizer used', 'Quantity of NPS fertilizer used,',
       'Quantity of 

In [446]:
fertilizer_qty_features = []
       
if year == "2010":
  fertilizer_qty_features =['If Chemical Fertilizer,Quantity in KG']

if year == "2011":
  fertilizer_qty_features = ['Quantity of chemical fertilizer used']

if year == "2012":  
  fertilizer_qty_features = ['Quantity of UREA chemical fertilizer used',
       'Quantity of DAP chemical fertilizer used',
       'Quantity of Both UREA and DAP chemical fertilizer used']

if year == "2013":
  fertilizer_qty_features = ["D22B", "D22C", "D22D", "D22E"]

if year in ("2014", "2015"):
  fertilizer_qty_features = ['Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
       'Quantity of UREA fertilizer used, if both DAP and UREA are used',
       'Quantity of DAP fertilizer used, if both DAP and UREA are used',]

if year == "2016":
  fertilizer_qty_features = ['Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
       'Quantity of UREA fertilizer used, if both DAP and UREA are used',
       'Quantity of DAP fertilizer used, if both DAP and UREA are used',
       'Quantity of OTHER fertilizer used',]

if year == "2017":
  fertilizer_qty_features = [ 'Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
       'Quantity of UREA fertilizer used, if both DAP and UREA are used',
       'Quantity of DAP fertilizer used, if both DAP and UREA are used',
       'Quantity of OTHER fertilizer used',
       'Quantity of UREA IF both UREA and NPS are used',
       'Quantity of NPS IF both UREA and NPS are used',]
       
if year == "2018":
  fertilizer_qty_features = [  'Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
       'Quantity of UREA fertilizer used, if both DAP and UREA are used',
       'Quantity of DAP fertilizer used, if both DAP and UREA are used',
       'Quantity of NPS fertilizer used',
       'Quantity of UREA IF both UREA and NPS are used',
       'Quantity of NPS IF both UREA and NPS are used',
       'Quantity of OTHER fertilizer used', ]
              
if year == "2019":
  fertilizer_qty_features = ['Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
       'Quantity of NPS fertilizer used,', 'Quantity of MIXED fertilizer used',
       'If natural fertilizer used, type', ]

if year == "2020":
  fertilizer_qty_features = ['Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
       'Quantity of NPS fertilizer used,',
       'Quantity of BLENDED fertilizer used',]

if year == "2021":
  fertilizer_qty_features = ['Quantity of UREA fertilizer used',
       'Quantity of DAP fertilizer used', 'Quantity of NPS fertilizer used,',
       'Quantity of BLENDED fertilizer used',]
       
fertilizer_qty_features

['Quantity of UREA fertilizer used',
 'Quantity of DAP fertilizer used',
 'Quantity of NPS fertilizer used,',
 'Quantity of BLENDED fertilizer used']

In [447]:
df[fertilizer_qty_features] = df[fertilizer_qty_features].fillna(0)

In [448]:
df["Fertilizer Quantity"] = df[fertilizer_qty_features].sum(axis=1)

In [449]:
df["Fertilizer Quantity"].describe()

count    109438.000000
mean         26.981411
std          54.221755
min           0.000000
25%           0.000000
50%           6.000000
75%          37.500000
max        6150.000000
Name: Fertilizer Quantity, dtype: float64

In [450]:
fert_threshold = df["Fertilizer Quantity"].quantile(.995)
fert_threshold

300.0

In [451]:
df = df[df["Fertilizer Quantity"] < fert_threshold]

In [452]:
df.shape

(108848, 43)

In [453]:
df = df.dropna(axis=1, thresh=df.shape[0]/2)
df.shape

(108848, 40)

In [454]:
df.columns

Index(['Region', 'Zone', 'District', 'Farmers Association', 'Enumeration Area',
       'Household Id', 'Head sex', 'Sampling weight', 'Field type', 'CROP',
       'Ownership type', 'Is field under Extension Program?',
       'Is Field Irrigated?', 'How many times is the field ploughed',
       'Orientation of the field', 'Is Field Prevented form Erosion',
       'If field Prevented form Erosion, common way of prevention',
       'Do youprevent land Degradation', 'Method to prevent land degredation',
       'Type of sowing', 'Seed / Seedling Type',
       'Quantity of indigenous seeds used', 'Was crop damaged ?',
       'Prevention/precaution measure taken?', 'Type of measure if any?',
       'Is Fertilizer Used?', 'Type of fertilizer used if any?',
       'Type of chemical fertilizer', 'Quantity of UREA fertilizer used',
       'Quantity of DAP fertilizer used', 'Quantity of NPS fertilizer used,',
       'Quantity of BLENDED fertilizer used',
       'How often is temporary crop field u

In [455]:
area = "Area in Square meter"
production = "Production Of Condition in Kg"
crop = "CROP"
region = "Region"
district = "District"

if year == "2013":
  area = "AREA"

if year == "2012":
  area = "Computed Area"
  production = "PRODUCTION"
  
if year == "2011":
  area = "Area in Hectare"
  production = "PRODUCTION IN QUINTALS"
  
if year == "2010":
  area = "AREAH"
  production = "PRODUCTION IN QUINTALS"
  
if year == "2009":
  area = "Area in hectare"
  production = "Production in Quintal"
  crop = "Crop or Landuse"
  
if year == "2008":
  area = "Area (sq. m.)"
  production = "Production in Quintal"
  crop = "Crop or Land Use"
  
if year == "2007":
  area = "Area in Hectare"
  production = "Production in Quintal"
  
if year == "2006":
  region = "Killil"
  district = "Wereda"
  area = "Area in hectar"
  production = "Production in Quintal"
    
if year == "2005":
  region = "Killil"
  district = "Wereda"
  area = "Area (sq. m.)"
  production = "Production in Quintal"
  crop = "Crop or Land Use"
    
if year == "2004":
  district = "District(Wereda)"
  area = "Area (sq. m.)"
  production = "Dry Weight Production (kg.)"
  crop = "Crop or Land Use"
    


In [456]:
df[[area]].describe()
df[[production]].describe()

,Production Of Condition in Kg
count,108777.000000
mean,539.852285
std,1269.172642
min,0.000000
25%,81.259000
50%,258.394000
75%,630.096000
max,144259.227000


In [457]:
min_threshold, max_threshold = df[[area]].quantile([0.05, 0.95]).values[:, 0]
min_threshold, max_threshold

(51.0, 6357.0)

In [458]:
df = df[(df[area] > min_threshold) & (df[area] < max_threshold)]

In [459]:
df.shape

(97938, 40)

In [460]:
df_valid = df.dropna(subset=[region, "Zone", district, crop, area, production])
df_valid.shape

(97880, 40)

In [461]:
df.nunique()

Region                                                              11
Zone                                                                21
District                                                            25
Farmers Association                                                 58
Enumeration Area                                                    24
Household Id                                                       248
Head sex                                                             2
Sampling weight                                                   2026
Field type                                                           2
CROP                                                                 5
Ownership type                                                       4
Is field under Extension Program?                                    2
Is Field Irrigated?                                                  2
How many times is the field ploughed                                 5
Orient

In [462]:
categorical_unique_value_limit = 100

df_zero = df_valid.fillna(0)
categorical_cols = []
numerical_cols = []

for col in df_zero.columns:
  if np.array_equal(df_zero[col], df_zero[col].astype("int")) and df[col].nunique() < categorical_unique_value_limit:
    categorical_cols.append(col)

    
for col in df_zero.columns:
  if not ( np.array_equal(df_zero[col], df_zero[col].astype("int")) and df[col].nunique() < categorical_unique_value_limit):
    numerical_cols.append(col)


print("Categorical Values (Nominal)")
print(categorical_cols, end="\n\n")

print("Categorical Values (Nominal)")
print(numerical_cols)


Categorical Values (Nominal)
['Region', 'Zone', 'District', 'Farmers Association', 'Enumeration Area', 'Head sex', 'Field type', 'CROP', 'Ownership type', 'Is field under Extension Program?', 'Is Field Irrigated?', 'How many times is the field ploughed', 'Orientation of the field', 'Is Field Prevented form Erosion', 'If field Prevented form Erosion, common way of prevention', 'Do youprevent land Degradation', 'Method to prevent land degredation', 'Type of sowing', 'Seed / Seedling Type', 'Was crop damaged ?', 'Prevention/precaution measure taken?', 'Type of measure if any?', 'Is Fertilizer Used?', 'Type of fertilizer used if any?', 'Type of chemical fertilizer', 'How often is temporary crop field used in Meher (main) season', 'What was the previous state of the field ?', 'Percent share of mixed crops']

Categorical Values (Nominal)
['Household Id', 'Sampling weight', 'Quantity of indigenous seeds used', 'Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used', 'Quantity of

In [463]:
df_valid.isna().sum()

Region                                                               0
Zone                                                                 0
District                                                             0
Farmers Association                                                  0
Enumeration Area                                                     0
Household Id                                                         0
Head sex                                                             0
Sampling weight                                                      0
Field type                                                           0
CROP                                                                 0
Ownership type                                                       0
Is field under Extension Program?                                    0
Is Field Irrigated?                                                  0
How many times is the field ploughed                                 0
Orient

In [464]:
from sklearn.impute import SimpleImputer

imputer = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")),
                    ("pandarizer",FunctionTransformer(lambda x: pd.DataFrame(x, columns = categorical_cols)))])
df_categorical = imputer.fit_transform(df_valid[categorical_cols])

imputer = Pipeline([("imputer", SimpleImputer(strategy="mean")),
                    ("pandarizer",FunctionTransformer(lambda x: pd.DataFrame(x, columns = numerical_cols)))])
df_numerical = imputer.fit_transform(df_valid[numerical_cols])

df_clean = df_categorical.join(df_numerical)

df_clean.isna().sum()

Region                                                           0
Zone                                                             0
District                                                         0
Farmers Association                                              0
Enumeration Area                                                 0
Head sex                                                         0
Field type                                                       0
CROP                                                             0
Ownership type                                                   0
Is field under Extension Program?                                0
Is Field Irrigated?                                              0
How many times is the field ploughed                             0
Orientation of the field                                         0
Is Field Prevented form Erosion                                  0
If field Prevented form Erosion, common way of prevention     

In [465]:
if year == "2013":
  df_clean.rename(columns = {'AREA':'Area in Square meter'}, inplace = True)
  df_clean

if year == "2012":
  df_clean.rename(columns = {'Computed Area':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'PRODUCTION':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] / 1000)
  df_clean

if year == "2011":
  df_clean.rename(columns = {'Area in Hectare':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'PRODUCTION IN QUINTALS':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
  df_clean

if year == "2010":
  df_clean.rename(columns = {'AREAH':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'PRODUCTION IN QUINTALS':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
  df_clean

if year == "2009":
  df_clean.rename(columns = {'Crop or Landuse':'CROP'}, inplace = True)
  df_clean.rename(columns = {'Area in hectare':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
  df_clean

if year == "2008":
  df_clean.rename(columns = {'Crop or Land Use':'CROP'}, inplace = True)
  df_clean.rename(columns = {'Area (sq. m.)':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean

if year == "2007":
  df_clean.rename(columns = {'Area in Hectare':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
  df_clean

if year == "2006":
  df_clean.rename(columns = {'Killil':'Region'}, inplace = True)
  df_clean.rename(columns = {'Wereda':'District'}, inplace = True)
  df_clean.rename(columns = {'Area in hectar':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
  df_clean

if year == "2005":
  df_clean.rename(columns = {'Killil':'Region'}, inplace = True)
  df_clean.rename(columns = {'Wereda':'District'}, inplace = True)
  df_clean.rename(columns = {'Crop or Land Use':'CROP'}, inplace = True)
  df_clean.rename(columns = {'Area (sq. m.)':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean

if year == "2004":
  df_clean.rename(columns = {'District(Wereda)':'District'}, inplace = True)
  df_clean.rename(columns = {'Crop or Land Use':'CROP'}, inplace = True)
  df_clean.rename(columns = {'Area (sq. m.)':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'Dry Weight Production (kg.)':'Production Of Condition in Kg'}, inplace = True)
  df_clean


In [466]:
df_1 = df_clean[df_clean["CROP"] == 1]
df_2 = df_clean[df_clean["CROP"] == 2]
df_6 = df_clean[df_clean["CROP"] == 6]
df_7 = df_clean[df_clean["CROP"] == 7]
df_8 = df_clean[df_clean["CROP"] == 8]

print(crop_1, df_1.shape)
print(crop_2, df_2.shape)
print(crop_6, df_6.shape)
print(crop_7, df_7.shape)
print(crop_8, df_8.shape)
print("\nTotal:", df_1.shape[0] + df_2.shape[0] + df_6.shape[0] + df_7.shape[0] + df_8.shape[0])

Barley (12024, 40)
Maize (31862, 40)
Sorghum (15642, 40)
Teff (23409, 40)
Wheat (14943, 40)

Total: 97880


In [467]:
df_1.to_csv(f"{path}Clean_Meher_{year}_{crop_1}.csv")
df_2.to_csv(f"{path}Clean_Meher_{year}_{crop_2}.csv")
df_6.to_csv(f"{path}Clean_Meher_{year}_{crop_6}.csv")
df_7.to_csv(f"{path}Clean_Meher_{year}_{crop_7}.csv")
df_8.to_csv(f"{path}Clean_Meher_{year}_{crop_8}.csv")

In [468]:
print(year)
print(2*f"{80*'* '}\n")

2021
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

